In [18]:
import pandas as pd
%pip install requests
import requests
# USGSから過去1か月の地震データを取得
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"
response = requests.get(url)

# データをファイルに保存
with open('earthquakes.csv', 'wb') as file:
    file.write(response.content)

# データをDataFrameに読み込む
data = pd.read_csv('earthquakes.csv')

# データの最初の5行を表示
data.head(100)

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2025-11-12T00:10:03.840Z,35.088000,-119.096500,12.2800,2.28,ml,62.0,25.0,0.03113,0.24,...,2025-11-12T00:13:35.333Z,"22 km NW of Grapevine, CA",earthquake,0.21,0.610000,0.192000,25.0,automatic,ci,ci
1,2025-11-12T00:07:01.005Z,54.616000,-160.283100,17.0000,3.60,ml,NaN,NaN,NaN,0.65,...,2025-11-12T00:21:01.811Z,"81 km S of Sand Point, Alaska",earthquake,NaN,0.400000,NaN,NaN,automatic,ak,ak
2,2025-11-11T23:40:28.942Z,32.073000,-101.617000,2.9722,1.20,ml,20.0,123.0,0.00000,0.40,...,2025-11-12T00:15:49.108Z,"17 km ESE of Stanton, Texas",earthquake,0.00,1.336424,0.300000,10.0,automatic,tx,tx
3,2025-11-11T23:39:44.307Z,61.717300,-151.675700,94.8000,2.00,ml,NaN,NaN,NaN,0.40,...,2025-11-11T23:52:11.356Z,"33 km SSW of Skwentna, Alaska",earthquake,NaN,0.500000,NaN,NaN,automatic,ak,ak
4,2025-11-11T23:38:46.270Z,19.347166,-155.046661,5.8800,1.88,ml,29.0,201.0,0.06118,0.28,...,2025-11-11T23:42:46.910Z,"16 km SE of Fern Forest, Hawaii",earthquake,0.99,0.870000,1.400000,3.0,automatic,hv,hv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2025-11-11T10:35:08.049Z,-19.395200,-69.145600,10.0000,4.80,mb,46.0,119.0,0.74200,0.86,...,2025-11-11T12:57:32.252Z,"30 km ESE of Camiña, Chile",earthquake,5.19,1.908000,0.083000,45.0,reviewed,us,us
96,2025-11-11T10:34:21.748Z,65.162500,-150.066500,14.0000,1.60,ml,NaN,NaN,NaN,0.49,...,2025-11-11T10:36:23.945Z,"32 km ENE of Manley Hot Springs, Alaska",earthquake,NaN,0.300000,NaN,NaN,automatic,ak,ak
97,2025-11-11T10:32:45.680Z,-19.388300,-69.222300,10.0000,4.60,mb,33.0,115.0,0.75200,1.32,...,2025-11-11T12:07:13.040Z,"22 km ESE of Camiña, Chile",earthquake,3.32,1.909000,0.101000,29.0,reviewed,us,us
98,2025-11-11T10:26:31.480Z,37.757668,-121.945335,7.4200,1.35,md,10.0,122.0,0.03100,0.10,...,2025-11-11T12:42:21.656Z,"4 km SE of San Ramon, CA",earthquake,0.43,1.050000,0.090000,9.0,automatic,nc,nc


In [28]:
data["country"] = data["place"].str.split(",").str[-1]
ranking = data["country"].value_counts().head(10)

ranking_df = ranking.reset_index()
ranking_df.columns = ["country", "count"]
ranking_df.index += 1

ranking_df["point"] = ranking_df["count"].rank(method="min", ascending=False).astype(int)
ranking_df["point"] = ranking_df["point"].max() - ranking_df["point"] + 1

ranking_mag = data.sort_values("mag", ascending=False)[["mag", "country"]].head(10)
ranking_mag = ranking_mag.reset_index(drop=True)
ranking_mag.index += 1
ranking_mag["point"] = ranking_mag.index.map(lambda x: 11 - x)

merged = pd.merge(ranking_df, ranking_mag[["country", "point"]], on="country", how="outer", suffixes=("_freq", "_mag"))

merged = merged.fillna(0)

merged["total_point"] = merged["point_freq"] + merged["point_mag"]

max_mag_per_country = data.groupby("country")["mag"].max().reset_index()
max_mag_per_country.columns = ["country", "max_mag"]

final_ranking = pd.merge(merged, max_mag_per_country, on="country", how="left")
final_ranking["total_point"] = final_ranking["total_point"].astype(int)
final_ranking["point_freq"] = final_ranking["point_freq"].astype(int)
final_ranking["point_mag"] = final_ranking["point_mag"].astype(int)

final_ranking = final_ranking.sort_values("total_point", ascending=False).reset_index(drop=True)

final_ranking.index += 1

print("\n\n=== 合計ポイント  頻度 + 最大マグニチュード ランキング ===\n")
print(final_ranking[["country", "count", "max_mag", "point_freq", "point_mag", "total_point"]].head(10))



=== 合計ポイント  頻度 + 最大マグニチュード ランキング ===

         country   count  max_mag  point_freq  point_mag  total_point
1          Japan     0.0     6.80           0         10           10
2             CA  2353.0     3.83           9          0            9
3      Indonesia     0.0     6.50           0          9            9
4     Guadeloupe     0.0     6.50           0          8            8
5         Alaska  1942.0     5.40           8          0            8
6          Texas   573.0     3.20           7          0            7
7          Japan     0.0     6.80           0          7            7
8      Banda Sea     0.0     6.40           0          6            6
9     Washington   196.0     2.85           6          0            6
10   Afghanistan     0.0     6.30           0          5            5
